# Ejercicio 2 - Mejoras y Comparación de Modelos de Clasificación

Este ejercicio considera completar acciones para mejorar el rendimiento de modelos de clasificación supervisada, (similar al ejercicio 1), pero se enfoca en realizar un análisis comparativo entre diferentes modelos utilizados para entender las ventajas/desventajas de unos y otros sobre este dataset y sus condiciones.

## Contexto: Análisis de éxito en campaña de marketing

Fuente: https://archive.ics.uci.edu/ml/datasets/Bank+Marketing

El foco está en la implementación de varios clasificadores para predecir el valor de un atributo de interés, desde un *dataset* de información de un resultados de personas contactadas por una campaña de marketing y que compraron la oferta (atributo "OK"), con cerca de 41.200 registros de personas contactadas.

Este conjunto de datos (abierto para este tipo de usos instruccionales), consiste en 20 atributos y 1 clase de etiquetas (totalizando 21 columnas) y corresponde a los datos de una campaña telefónica a diversos clientes en Portugal, ofreciéndoles la compra de un producto bancario. En varios casos, un cliente fue contactado varias veces antes de aceptar el depósito a plazo ofrecido por la campaña (OK = yes).

Algunos de los atributos relevantes son (combinando atributos categóricos, con numéricos):
* **Datos personales**: Edad, Ocupación, Estado Civil, Nivel de Educación.
* **Datos financieros**: Su casa tiene crédito hipotecario, default: si el crédito ha caído en quiebra; tiene un crédito de consumo.
* **Datos de contactos de la campaña actual**: Tipo de Comunicación (celular o teléfono fijo); Mes del último contacto; Día de la semana del contacto; duración de la llamada (segundos); Contacto: N° de contactos durante la campaña; DíasAtrás: días transcurridos desde último contacto; Resultado: resultado de la última llamada (falló, no-existe, éxito)
* **Datos socioeconómicos**: EmpTasaVar: tasa de variación de empleabilidad; IPC: índice de precios consumidor mensual; ICC: índice de confianza consumidor mensual; Euribor3m: tasa euribor de 3 meses indicador diario; NumEmpleados: cantidad de gente empleada, en indicador trimestral.

Esta adaptación en particular, por el equipo de R:Solver (RSolver.com), enfrenta diferentes objetivos de aprendizaje dentro de los cursos de Big Data y Machine Learning.



## Instrucciones Generales
En este caso, se busca entender el comportamiento y desempeño de diferentes modelos de clasificación sobre este conjunto de datos, para predecir la variable de interés: **OK**, que servirá para predecir en casos futuros, según los datos de contactabilidad de un cliente, si el cliente aceptará o no contratar el depósito a plazo.

La entrega (grupal o individual, según corresponda) se materializa en un informe donde se contestan las preguntas que se indican en las secciones de "Preguntas", más adelante. Se puede recurrir a ejercicios de otras fuentes, así como al material de clases.

Dentro del informe se puede considerar una tabla de datos de ejecuciones comparadas de los modelos y con diferentes condiciones (balance de clases, proporciones de % entrenamiento-evaluación), apoyando las respuestas a las preguntas correspondientes.

La entrega se realiza en forma de un **informe en formato PDF, adjunto por email** utilizando la plantilla de informe que está en http://dcc.rsolver.com/dcc/docs/InformeActividad.docx

El informe en formato PDF debe ser subido por sólo uno de los integrantes a la siguiente URL

http://aiker.ai/aiker/DocUpload.aspx (*)

(*)Si hay problemas en la carga, enviar el PDF a rsandova@ing.puc.cl y cc: ayudante@aiker.ai

## Paso 1: Instación de las librerías de modelos de clasificación

Esto se ejecuta sólo una vez al comienzo de la sesión de cada persona. No se necesita volver a ejecutar con cada nueva prueba del resto de los scripts. Aquí se incluyen liberías para ejecutar todos los modelos.

In [ ]:
install.packages('e1071')
install.packages('caret')
install.packages('rpart')
install.packages('rpart.plot')
install.packages('randomForest')
install.packages('class')
install.packages("nnet")

##Paso 2: Carga y preprocesamiento de los datos ##

A continuación se cargan el conjunto de datos desde la URL de origen. Esta versión preprocesa el dataset, realizando actividades de limpieza de datos, eliminando filas con algún NA y eliminando unas pocas columnas que se determinan como no-relevantes en el desempeño de modelos de clasificación.

Adicionalmente, se realiza un balance entre las clases, reduciendo la cantidad de ejemplos de la clase mayoritaria para aproximarse a la otra.

Se entiende que con estas acciones de preprocesamiento del dataset, ya se llega en mejores condiciones a este nuevo ejercicio.

Este código se puede ejecutar sólo una vez para uso del dataset resultante en los siguientes pasos.

In [ ]:
# Se declara la URL de dónde obtener los datos
theUrlMain <- "http://www.rsolver.com/dcc/docs/bank-additional-full.csv"

# Se declaran los nombres de las columnas
columnas = c("Edad","Ocupación","EstadoCivil","Educación","Default","Hipotecario","Consumo","Contacto","Mes","Día",
             "Duración","NumContactos","DíasAtrás","Previo","ResultadoPrevio",
             "EmpTasaVar", "IPC", "ICC", "Euribor3m", "NumEmpleados", "OK")

# Se cargan datos principales a una estructura o dataset (marketing.data), asignando nombres de atributos a las columnas.
# Nótese que se incluye la conversión de valores "unknown" a "NA" para facilitar la gestión vacíos más adelante.
marketing.data <- read.table(file = theUrlMain, header = TRUE, sep = ";", col.names = columnas, na.strings=c("unknown","NA"))

# Se eliminan aquellos atributos que no aportan en el desempeño de los modelos
# Esto se determinó en un trabajo previo, fuera de esta actividad
marketing.data$Default <- NULL
marketing.data$DíasAtrás <- NULL
marketing.data$Previo <- NULL
marketing.data$Euribor3m <- NULL

# Se eliminan los registros que tienen algún NA (antes: 'unknown')
# Nótese que se hace esta limpieza posterior a la eliminación de columnas,
# logrando conservar la enorme mayoría de los registros.
marketing.clean <- na.omit(marketing.data)
dim(marketing.clean) # Sólo quedan poco más de 38.000 filas (de las 41.000 originales)

# Se muestran las primeras líneas del dataset, incluyendo sólo las columnas que quedaron.
head(marketing.clean, 20)
dim(marketing.clean)

# Aquí se arman dos subconjuntos con los datos de cada una de las dos clases.
# Se pueden ver los respectivos tamaños al terminar, evidenciando un desbalance.
clean.data.YES <- marketing.clean[marketing.clean$OK == 'yes',]
clean.data.NO <- marketing.clean[marketing.clean$OK == 'no',]
cat("Cantidad de ejemplos por clase\n")
dim(clean.data.YES) # Este es el conjunto más pequeño con poco más de 4.000 ejemplos
dim(clean.data.NO)  # Este es mayoritario con casi 34.000 ejemplos, evidenciando desbalance entre clases

# A continuación se realiza un re-balanceo de las clases (random subsampling),
# que consiste en reducir la cantidad de ejemplos de la clase más masiva, para acercarla a la minoritaria.
balance_ratio <- 1.2 # Se elige un balanceo de 20% más de ejemplos de la clase negativa que la positiva

clean.subdata.YES <- clean.data.YES  # No se aplica sample(); se usan todos los ejemplos de la clase YES (que es la que tiene menos ejemplos)
clean.subdata.NO <- clean.data.NO[sample(nrow(clean.data.NO), balance_ratio*dim(clean.data.YES)[1]), ] # Se elige un subconjunto de los NO

# Muestra cantidad de ejemplos contenidos en cada subconjunto
cat("Cantidad de ejemplos por clase luego del balance entre clases\n")
dim(clean.subdata.YES)
dim(clean.subdata.NO)

# Se juntan para el conjunto de referencia, ahora más balanceado
clean.subdata <- rbind(clean.subdata.YES, clean.subdata.NO)

summary(clean.subdata)


## Ejercicio 1: Preparación conjuntos de entrenamiento y evaluación

En este caso, lo importante es considerar que **es posible cambiar la proporción de datos de entrenamiento y test** viendo el efecto que tiene en el desempeño de los modelos (del ejercicio 2: RandomForest, SVM, NB), viendo que alguno o varios de los modelos entregan mejores resultados considerando lo que más puede interesar, entre Accuracy, Sensitivity, Specificity.

**Pregunta 1** (1.5 puntos)

¿Cuál es la proporción entrenamiento/test que logra mejor desempeño y con cuál de los modelos entre RandomForest, SVM, NB?

El objetivo es probar varias combinaciones cambiando la proporción de datos. Preliminarmente 4 combinaciones, desde 60%/40% hasta 90%/10%. Según los resultados de la ejecución de todos los modelos de clasificación más adelante, determinar y explicitar cuál es la proporción que logra mejores resultados o desmpeño de clasificación y cuál es la influencia del cambio de proporción de entrenamiento/test. Particularmente se espera que los alumnos determinen cómo se elige el mejor modelo (comparando Sensitivity, Specificity, Accuracy).

Se pide documentar en una tabla todas las combinaciones, viendo los indicadores de desempeño más relevantes: Accuracy, Sensitivity, Specificity, **determinando cuál combinación da mejores resultados para cuál de los modelos** (RandomForest, NB, SVM), considerando que el desempeño se logra por maximizar el desempeño de la predicción de YES, además de un buen modelo balanceado (accuracy).


In [ ]:
# Primero, se saca una copia del dataset para trabajar sin modificar el original
# Esto permite hacer más modificaciones y correr este código varias veces sin alterar clean.subdata
working.data <- clean.subdata

# EJERCICIO 1
# Ahora se configuran los conjuntos de entrenamiento y testing en una proporción
# (por ej: 0.70 = 70% para training y el resto para evaluación o testing)
# Se pide probar diferentes combinaciones (60/40, 70/30, 80/20, 90/10)
# hasta determinar cuál es la mejor en cuál de los modelos.
ratio = sample(1:nrow(working.data), size = 0.80*nrow(working.data))
training.data = working.data[ratio,]
testing.data = working.data[-ratio,]

# Se comparan los tamaños de ejemplos para entrenamiento y evaluación.
dim(training.data)
dim(testing.data)

head(training.data)

## Ejercicio 2: Comparación de desempeño de modelos de clasificación y su explicación

Habiendo definido los conjuntos de entrenamiento y de test, a continuación se ejecutan unos modelos de clasificación: un RandomForest, un Naive Bayes, y un Support Vector Machine. Cada uno obtiene sus resultados, mostrando sus precisiones en desempeño. No es necesario modificar estos bloques de código. Basta con hacer los cambios en la parte del ejercicio 1 (proporción entrenamiento/test) y volver a ejecutar estos modelos para evaluar su desempeño.

Una vez completado el ejercicio 1 anterior (habiendo quedado con una ejecución de mejor desempeño y habiendo realizado la comparación de los indicadores), se pueden contestar las preguntas a continuación, que se centran en interpretar y analizar comparativamente del desempeño de estos modelos.

**Pregunta 2.1** (1 punto)

Viendo que un balance de clases de 1.2 (sólo un 20% más de ejemplos de la clase negativa sobre la positiva), donde reduce notoriamente la cantidad de ejemplos de la clase negativa, ¿por qué considera que se logra esa mejoría, a pesar de eliminar de entrenamiento y evaluación esa cantidad de ejemplos originales? (Justifique con claridad, según lo que se conoce sobre la forma en que se entrenan los modelos).

**Pregunta 2.2** (1 punto)

Habiendo determinado en el ejercicio 1 cuál es el modelo que tiene mejor desempeño entre todos, con una mejor proporción de entrenamiento/test ¿qué características del modelo apoyan su mejor desempeño sobre los otros modelos, aunque la diferencia haya sido menor? (Justifique con claridad, según lo que se conoce sobre las características particulares de los modelos y por qué ese modelo muestra mejor desempeño que los otros).

Recuerde que sólo se comparan los 3 modelos a continuación.


**Random Forest**

In [ ]:
library(randomForest)
library(caret)

# Random Forest
RF_model <- randomForest(as.factor(OK) ~ ., data=training.data, method="class")
RF_predict <- predict(RF_model, testing.data, type = "class")
confusionMatrix(RF_predict, as.factor(testing.data$OK), positive = 'yes')

**Naive Bayes**

In [ ]:
library(e1071)

# Naive Bayes
NB_model <- naiveBayes(as.factor(OK) ~ ., data=training.data)
NB_predict <- predict(NB_model, testing.data, type = "class")
confusionMatrix(NB_predict, as.factor(testing.data$OK), positive = 'yes')

**Support Vector Machine**

In [ ]:
library(e1071)

# Support Vector Machine (NOTA: toma algunos minutos su ejecución)
SVM_model <- svm(as.factor(OK) ~ ., data = training.data, cost = 10, scale = FALSE)
SVM_predict <- predict(SVM_model, testing.data, type = "class")
confusionMatrix(SVM_predict, as.factor(testing.data$OK), positive = 'yes')


# Ejercicio 3: Implementación de una Red Neuronal
A continuación se declara, entrena y evalúa un modelo de Red Neuronal. Esta primera declaración viene con una configuración inicial, que se podrá modificar para ver posibles mejoras en el desempeño de esta red.

Esta configuración considera lo siguiente:

*     **Nótese que sólo se utilizan algunos atributos del dataset**, que vienen en la declaración de la fórmula (1er argumento) de nnet(). Se pueden eliminar algunos y ver si mejora el desempeño.

*     Se usa una única capa escondida o intermedia. Su cantidad de nodos está dada por el atributo 'size'. Se puede agrandar o reducir para ver posibles mejoras.

*     La cantidad de iteraciones para mejorar el entrenamiento se da por el atributo 'maxit'. Se puede aumentar, esperando mejorar el desempeño.

*     El atributo 'maxNWts' limita el tamaño interno de la red, que dadas las restricciones de capacidad de procesamiento que entrega Google Colab, conviene acotarlo, para evitar sobrepasar la memora y tener una ejecución fallida. No es necesario modificar este atributo.

Hay otros atributos posibles de analizar y modificar en https://www.rdocumentation.org/packages/nnet/versions/7.3-14/topics/nnet. Nótese que la configuración por defecto usa una activación logística, pero es posible aplicar softmax o linout, pero eso requiere de parámetros adicionales.

Ojo/recomendación: dada la naturaleza aleatoria del comportamiento del entrenamiento, en ocasiones la red neuronal no entrega resultados para la clase menos representada y genera un error. En cuyo caso, sólo basta con volver a ejecutar el código, para que - aleatoriamente - logre dar resultados en dicha clase.

**Ejercicio 3:**

Probar diferentes versiones del modelo, cambiando:
*     Los atributos considerados. Por simplicidad se recomienda sólo eliminar algunos de la lista original, para ver si en alguna ejecución esa eliminación genera mejores resultados.
*     La cantidad de nodos de la capa escondida (size).
*     La cantidad de iteraciones (maxit).

Por simplicidad de este ejercicio, se recomienda sólo probar 4 combinaciones de cada uno de los 3 elementos a cambiar. Se pueden elegir los valores de esos cambios y documentar en una tabla de ejecuciones comparadas para contestar la pregunta 3.1.

**Preg 3.1** (1.8 puntos): ¿Cuáles son los parámetros de ejecución del modelo que dan el mejor desempeño de la Red Neuronal?

**Preg 3.2** (0.7 puntos): ¿Logra superar al mejor modelo de los primeros 3 modelos? ¿Por qué considera que si o no y qué caracteristica distinta entre estos 2 modelos hace la diferencia? (En cualquier caso, se pide una posible y teórica explicación de por qué es mejor/peor que ese otro modelo.)

In [ ]:
library(nnet)

# Red Neuronal
# Lista original de atributos: Edad+Ocupación+EstadoCivil+Educación+Duración+NumContactos+EmpTasaVar+NumEmpleados
NN_model <- nnet(as.factor(OK) ~ Edad+Ocupación+EstadoCivil+Educación+Duración+NumContactos+EmpTasaVar+NumEmpleados,
                  data=training.data, size=25, maxit=3000, MaxNWts=10000)
NN_predict <- predict(NN_model, testing.data, type="class")

# A continuación se muestra el resultado de evaluación
cat("Resultados Red Neuronal\n")
confTable <- table(NN_predict, testing.data$OK)
confTable

accuracy <- (confTable[1,1] + confTable[2,2]) / dim(testing.data)[1]
cat("\nAccuracy:    ", accuracy)

sensitivity <- confTable[1,1] / (confTable[1,1] + confTable[1,2])
cat("\nSensitivity: ", sensitivity)

specificity <- confTable[2,2] / (confTable[2,1] + confTable[2,2])
cat("\nSpecificity: ", specificity)



---

**Complemento: Ejercicio de comprobación manual**


Para verificar que alguno de los modelos realmente predice correctamente, se comprueba con los datos de una persona en particular, pidiendo la predicción al modelo. A continuación hay dos ejemplos, que se pueden modificar para ver su resultado, cambiando valores y también, cambiando el modelo a utilizar en la predicción. No se necesita modificar, ni comentar esta parte en la entrega, sino que se entrega como complemento para quienes tengan el interés de ver cómo se aplica un modelo entrenado en un contexto práctico (en producción)

In [ ]:
# Ejemplo 1: La predicción debería ser "YES"
sample_x <- clean.subdata[1,]
sample_x[1,1] <- 32       # Edad
sample_x[1,2] <- 'admin.' # Ocupación
sample_x[1,3] <- 'single' # EstadoCivil
sample_x[1,4] <- 'university.degree'  # Educación
sample_x

prediction <- predict(RF_model, sample_x)
prediction


# Ejemplo 2: La predicción debería ser "NO"
sample_x2 <- clean.subdata[1,]
sample_x2[1,10] <- 10   # Duración
sample_x2

prediction <- predict(RF_model, sample_x2)
prediction


,Edad,Ocupación,EstadoCivil,Educación,Hipotecario,Consumo,Contacto,Mes,Día,Duración,NumContactos,ResultadoPrevio,EmpTasaVar,IPC,ICC,NumEmpleados,OK
,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
76,32,admin.,single,university.degree,yes,no,telephone,may,mon,1575,1,nonexistent,1.1,93.994,-36.4,5191,yes


76 
yes 
Levels: no yes

,Edad,Ocupación,EstadoCivil,Educación,Hipotecario,Consumo,Contacto,Mes,Día,Duración,NumContactos,ResultadoPrevio,EmpTasaVar,IPC,ICC,NumEmpleados,OK
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
76,41,blue-collar,divorced,basic.4y,yes,no,telephone,may,mon,10,1,nonexistent,1.1,93.994,-36.4,5191,yes


76 
no 
Levels: no yes